In [1]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
AURA_INSTANCENAME=os.environ["AURA_INSTANCENAME"]
NEO4J_URI=os.environ["NEO4J_URI"]
NEO4J_USERNAME=os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD=os.environ["NEO4J_PASSWORD"]
NEO4J_DATABASE=os.environ["NEO4J_DATABASE"]
AUTH=(NEO4J_USERNAME, NEO4J_PASSWORD)

In [3]:
driver=GraphDatabase.driver(NEO4J_URI, auth=AUTH, database=NEO4J_DATABASE)

In [6]:
def connect_and_query():
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            result=session.run("MATCH (n) RETURN count(n)")
            count=result.single().value()
            print(f"Number of nodes: {count}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

In [7]:
connect_and_query()

C:\Users\varsh\AppData\Local\Temp\ipykernel_18516\3985270811.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database=NEO4J_DATABASE) as session:


Number of nodes: 5


In [23]:
def create_entities(tx):
    tx.run("MERGE (a:Person {name: 'Albert Einstein'})")
    
    tx.run("MERGE (p:Subject {name: 'Physics'})")
    tx.run("MERGE (n:NobelPrize {name: 'Nobel Prize in Physics'})")
    tx.run("MERGE (g:Country {name: 'Germany'})")
    tx.run("MERGE (u:Country {name: 'USA'})")

In [21]:
def create_relationships(tx):
    tx.run(
        """
        MATCH (a:Person{name: 'Albert Einstein'}), (p: Subject {name:'Physics'})
        MERGE (a)-[:STUDIED]->(p)
        """
    )
    
    tx.run(
        """
        MATCH (a:Person{name: 'Albert Einstein'}), (n: NobelPrize {name:'Nobel Prize in Physics'})
        MERGE (a)-[:WON]->(n)
        """
    )
    
    tx.run(
        """
        MATCH (a:Person{name: 'Albert Einstein'}), (g: Country {name:'Germany'})
        MERGE (a)-[:BORN_IN]->(g)
        """
    )
    
    tx.run(
        """
        MATCH (a:Person{name: 'Albert Einstein'}), (u: Country {name:'USA'})
        MERGE (a)-[:DIED_IN]->(u)
        """
    )

In [9]:
#Function to connect and run a simple cypher query
def query_graph_simple(cypher_query):
    driver=GraphDatabase.driver(NEO4J_URI, auth=AUTH)
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            result=session.run(cypher_query)
            for record in result:
                print(record["name"])
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

In [10]:
#Function to connect and run a simple cypher query
def query_graph(cypher_query):
    driver=GraphDatabase.driver(NEO4J_URI, auth=AUTH)
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            result=session.run(cypher_query)
            for record in result:
                print(record["path"])
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

In [18]:
def build_knowledge_graph():
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            session.execute_write(create_entities)
            session.execute_write(create_relationships)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

In [24]:
build_knowledge_graph()

C:\Users\varsh\AppData\Local\Temp\ipykernel_24512\2974030905.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database=NEO4J_DATABASE) as session:


In [11]:
simple_query="""
MATCH (n) RETURN n.name AS name
"""

query_graph_simple(simple_query)

Albert Einstein
Physics
Nobel Prize in Physics
Germany
USA
